In [4]:
!pip install dspy --quiet
!pip install fastapi uvicorn --quiet

In [5]:
import dspy
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))
dspy_program = dspy.ChainOfThought("question -> answer")

In [6]:
import pandas as pd

# Loaded variable 'df' from URI: c:\Users\LNshuti\OneDrive - Sony\Desktop\learn\usgov-contracts-rag\data\sample_contract_df.parquet
df = pd.read_csv('../data/fy2022_archived_opportunities.csv', encoding='latin-1')
df.head()

/var/folders/xz/9d0gd46n5lgfq9x35p2g02sw0000gn/T/ipykernel_96363/1440626228.py:4: DtypeWarning: Columns (17,27,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/fy2022_archived_opportunities.csv', encoding='latin-1')


,NoticeId,Title,Sol#,Department/Ind.Agency,CGAC,Sub-Tier,FPDS Code,Office,AAC Code,PostedDate,...,SecondaryContactPhone,SecondaryContactFax,OrganizationType,State,City,ZipCode,CountryCode,AdditionalInfoLink,Link,Description
0,97ec6ab3d5d4454192a8828827de6f2f,Whooster Web-Based Software,HQ003422P0171,DEPT OF DEFENSE,97.0,WASHINGTON HEADQUARTERS SERVICES (WHS),97F5,WASHINGTON HEADQUARTERS SERVICES,HQ0034,2022-09-30 23:09:21.289-04,...,NaN,NaN,OFFICE,DC,WASHINGTON,203011000,USA,NaN,https://sam.gov/opp/97ec6ab3d5d4454192a8828827...,The Whooster Web Based Software contract is a ...
1,c15a7148317b44eeac210a7b64068765,Beckman Coulter Reagents and Supplies for Gove...,W81K0222Q0162,DEPT OF DEFENSE,21.0,DEPT OF THE ARMY,2100,W40M RHCO PACIFIC,W81K02,2022-09-30 23:01:45.502-04,...,NaN,NaN,OFFICE,HI,FORT SHAFTER,96858-5098,USA,NaN,https://sam.gov/opp/c15a7148317b44eeac210a7b64...,The Medical Readiness Contracting Office  Pac...
2,bf103afa7b4b455bb1541eb450e38128,8(a) WebEOC IL5 Cloud Services and IT Support,HQ003422C0130,DEPT OF DEFENSE,97.0,WASHINGTON HEADQUARTERS SERVICES (WHS),97F5,WASHINGTON HEADQUARTERS SERVICES,HQ0034,2022-09-30 22:12:14.082-04,...,NaN,NaN,OFFICE,DC,WASHINGTON,203011000,USA,NaN,https://sam.gov/opp/bf103afa7b4b455bb1541eb450...,Direct Award of an 8(a) contract for WebEOC IL...
3,d1619b0084ac4d439585b727343b4659,Leslie Valve IDIQ - Sources Sought,N4523A22RC04N81,DEPT OF DEFENSE,17.0,DEPT OF THE NAVY,1700,PUGET SOUND NAVAL SHIPYARD IMF,N4523A,2022-09-30 22:05:45.599-04,...,3609793943,NaN,OFFICE,WA,BREMERTON,98314-5001,USA,NaN,https://sam.gov/opp/d1619b0084ac4d439585b72734...,Puget Sound Naval Shipyard and Intermediate Ma...
4,48b6b27846bd4155990cdc2f2be335c5,PFPA BALLISTIC VEST AND PLATES,HQ003422Q0206,DEPT OF DEFENSE,97.0,WASHINGTON HEADQUARTERS SERVICES (WHS),97F5,WASHINGTON HEADQUARTERS SERVICES,HQ0034,2022-09-30 21:36:36.954-04,...,NaN,NaN,OFFICE,DC,WASHINGTON,203011000,USA,NaN,https://sam.gov/opp/48b6b27846bd4155990cdc2f2b...,This blanket purchase agreement (BPA) has been...


In [7]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

import dspy

app = FastAPI(
    title="DSPy Program API",
    description="A simple API serving a DSPy Chain of Thought program",
    version="1.0.0"
)

# Define request model for better documentation and validation
class Question(BaseModel):
    text: str

# Configure your language model and 'asyncify' your DSPy program.
lm = dspy.LM("openai/gpt-4o-mini")
dspy.settings.configure(lm=lm, async_max_workers=4) # default is 8
dspy_program = dspy.ChainOfThought("question -> answer")
dspy_program = dspy.asyncify(dspy_program)

@app.post("/predict")
async def predict(question: Question):
    try:
        result = await dspy_program(question=question.text)
        return {
            "status": "success",
            "data": result.toDict()
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [8]:
import dspy
import mlflow
import requests

mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [9]:
!curl http://127.0.0.1:6000/invocations -H "Content-Type:application/json"  --data '{"messages": [{"content": "what is 2 + 2?", "role": "user"}]}'

curl: (7) Failed to connect to 127.0.0.1 port 6000 after 0 ms: Couldn't connect to server


In [10]:
!mlflow models build-docker 
!docker run -p 6000:8080 dspy-program

2024/12/07 20:41:06 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/12/07 20:41:06 INFO mlflow.pyfunc.backend: Building docker image with name mlflow-pyfunc-servable
Traceback (most recent call last):
  File "/opt/miniconda3/envs/gov-data/lib/python3.11/site-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/gov-data/lib/python3.11/site-packages/urllib3/connectionpool.py", line 495, in _make_request
    conn.request(
  File "/opt/miniconda3/envs/gov-data/lib/python3.11/site-packages/urllib3/connection.py", line 398, in request
    self.endheaders()
  File "/opt/miniconda3/envs/gov-data/lib/python3.11/http/client.py", line 1298, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/miniconda3/envs/gov-data/lib/python3.11/http/client.py", line 1058, in _send_output
    self.send(msg)
  File "/opt/m